# 基于Llama2的混合索引

## 数据获取(以马拉色菌为case)

In [ ]:
import pandas as pd
df_seq=pd.read_csv('./invention_processed.csv')
patent_kind='A1'  #B
patent_num='20200277334'  #"09169290"  09273096 11684559  09265709  20220062139 10329336
patent_country='US'

In [2]:
import random

def shuffle_seq(seq_list):
#用于随机排列seq
    random.shuffle(seq_list)
    return ''.join(seq_list)

#对于sequence数量过多的patent，将其按一定间隔分段输入prompt
def cut_prompts(df,total_length=1000,max_num=30,rand_num=5):
    #设置最大prompt长度和最大seq数量，防止prompt占用过多token
    prompts=[]
    seq_set=[]
    len_now=0
    num_now=0
    for index,row in df.iterrows():
        new_str='{}:{}'.format(row['new_seq_id'],row['peptide sequence'])
        if(len_now+len(new_str)<=total_length and num_now+1<=max_num):
            #长度和数量都满足时加入prompt
            pass
        else:
            #有一项不满足就新增prompt,并将一组seq随机排列多次
            prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
            seq_set=[]
            len_now=0
            num_now=0

        len_now+=len(new_str)
        num_now+=1
        seq_set.append(new_str)
        
    prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
    return prompts

In [3]:
df_unit=df_seq[df_seq['patent_no']==patent_country+patent_num+patent_kind]
prompts=cut_prompts(df_unit,max_num=1,rand_num=1)
prompts

[['SEQ_ID_NO_18:WRRWWRRWWRWWRRWWRR'],
 ['SEQ_ID_NO_17:RWWRWWRRWWRR'],
 ['SEQ_ID_NO_19:RRVVRRVRRVVRRVVRVVRRVVRR'],
 ['SEQ_ID_NO_21:VRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR'],
 ['SEQ_ID_NO_20:RRWWRRWRRWWRRWWRWWRRWWRR'],
 ['SEQ_ID_NO_23:RRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR'],
 ['SEQ_ID_NO_22:VRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWVRR'],
 ['SEQ_ID_NO_25:RVVRVVRRWVRRVRRVWRRVVRVVRRWVRRVRRVWRRVVRVVRRWRVV'],
 ['SEQ_ID_NO_24:RVVRVVRRVVRRVRRVVRRVVRVVRRVVRRVRRVVRRVVRVVRRVVRR'],
 ['SEQ_ID_NO_26:HHHHHH'],
 ['SEQ_ID_NO_2:IRRRRRRIRRRRRR'],
 ['SEQ_ID_NO_3:IRRRIRRIRRRIRRIRRRIRR'],
 ['SEQ_ID_NO_4:IRRIIRRIRRIIRRIRRIIRR'],
 ['SEQ_ID_NO_5:VWRWVRRVWRWVRRVWRWVRR'],
 ['SEQ_ID_NO_6:VWRWVRRVWRWVRR'],
 ['SEQ_ID_NO_7:VVRVVRRVVRVVRR'],
 ['SEQ_ID_NO_8:VVRVVRVVVRVVRVVVRVVRV'],
 ['SEQ_ID_NO_9:RSRVVRSWSRV'],
 ['SEQ_ID_NO_1:RRWVRRVRRVWRRVVRVVRRWVRR'],
 ['SEQ_ID_NO_10:RFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVRRFVRRVR'],
 ['SEQ_ID_NO_12:KVVSSIIEIISSVVKVVSSIIEIISSVV'],
 ['SEQ_ID_NO_11:RRTYSR

In [4]:
from elasticsearch import Elasticsearch

import json

es = Elasticsearch(hosts="http://192.168.1.124:29200")
INDEX = "patents"

body = {
    "query":{
        "bool":{
            "must":[
                {"term":{"publication_doc_number": patent_num}},
                {"term":{"publication_country": patent_country}}, 
                {"term":{"publication_kind":patent_kind}}               
            ]
        }
    }
}

result = es.search(index=INDEX,body=body, request_timeout=60)
#从es中获取原文
res_dic=dict(result)
res_js=json.dumps(dict(result),indent=2)

/tmp/ipykernel_2392450/1067941293.py:20: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_2392450/1067941293.py:20: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


In [5]:
print(res_js)

{
  "took": 984,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 13.942645,
    "hits": [
      {
        "_index": "patents",
        "_type": "_doc",
        "_id": "0PJFgYwB_RI5Z0p9zZZF",
        "_score": 13.942645,
        "_ignored": [
          "description.Reference.keyword",
          "description.Summary.keyword",
          "description.Others.keyword",
          "description.Description.keyword"
        ],
        "_source": {
          "patent_type": "application",
          "invention": "ENGINEERED ANTIMICROBIAL AMPHIPHILIC PEPTIDES AND METHODS OF USE",
          "application_country": "US",
          "application_doc_number": "16801782",
          "application_date": "2020-02-26",
          "publication_country": "US",
          "publication_doc_number": "20200277334",
          "publication_kind": "A1",
          "pu

## llama2-chat模型加载

### 方式一：加载本地llama2模型

In [6]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


from IPython.display import Markdown, display

In [7]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

LLAMA2_7B_chat = "/home/cymei/llama_hf/llama-2-chat-7b-hf"
LLAMA2_13B_chat = "/mnt/sdc/Llama-2-13b-chat-hf"
LLAMA2_7B_hf = "/home/cymei/llama_hf/llama-2-7b-hf"

selected_model = LLAMA2_7B_chat #LLAMA2_7B_chat

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:107: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=118


  warn((f'\n\n{"="*80}\n'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 方式二：使用llama2自己的API

In [5]:
#需要去https://www.llama-api.com/注册
#调用api收费
from llama_index.llms.llama_api import LlamaAPI
api_key = "your API"
llm = LlamaAPI(api_key=api_key)

## 对输入文本进行预处理，并将大模型设置为全局服务

In [6]:
import re
from llama_index.schema import Document

claims=['claim '+' '.join(dic['claim_text']) for dic in res_dic['hits']['hits'][0]['_source']["claim"]]
claims_raw='The claim information:\n'+'\n'.join(claims)
description_raw='\nThe description information:\n'+'\n'.join(res_dic['hits']['hits'][0]['_source']["description"]["Description"])

#文本预处理(方便之后的关键词匹配)
claims1=re.sub('\u2003',' ',claims_raw)
claims_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',claims1)
description1=re.sub('\u2003',' ',description_raw)
description_real=re.sub('SEQ ID N[oO][.: ]+','SEQ_ID_NO_',description1)

#claim和description存储在不同的document中
doc_claim = Document(text=claims_real)
doc_des = Document(text=description_real)

In [7]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    set_global_service_context,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#加载RAG需要的embedding模型
embed_model = HuggingFaceEmbedding(
    model_name="/home/cymei/bge-small-en",device='cuda:0'
)

#设置全局服务
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model#"local:/home/cymei/bge-small-en"
)
set_global_service_context(service_context)


/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 使用简单切片或层次切片作为混合索引的基础

In [8]:
from llama_index.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
# parse nodes
# parser = SentenceSplitter(chunk_size=256,chunk_overlap=8) #控制每个切片的长度以及相邻切片的重叠长度
# nodes = parser.get_nodes_from_documents([doc_claim,doc_des])

node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[1024, 512, 256])
nodes = node_parser.get_nodes_from_documents([doc_claim,doc_des]) #documents

In [9]:
# define storage context
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage import StorageContext

docstore = SimpleDocumentStore()
# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [10]:
#层次索引需要载入底层的叶子节点
from llama_index.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(nodes)
print(len(leaf_nodes))
root_nodes = get_root_nodes(nodes)
print(len(root_nodes))

258
49


## 混合索引

In [11]:
## Load index into vector index
from llama_index import VectorStoreIndex

# 利用叶子节点计算相似度，并关联到对应的父节点
base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)


In [12]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
)
from llama_index.schema import BaseNode

from typing import List

In [13]:
from nltk import word_tokenize 

class CustomRetriever(BaseRetriever):
    """自定义的混合索引类：相似度索引+关键词索引"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        vector_retriever_large: VectorIndexRetriever,
        max_num_keyword_nodes=3,
        keywords=[]
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever #头部相似度节点
        self._vector_retriever_large = vector_retriever_large #更大范围的头部相似度节点(用于关键词搜索)

        self.keywords=keywords #索引依据的关键词
        self.max_num_keyword_nodes=max_num_keyword_nodes #设置最大关键词节点数量
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        #利用两个不同的参数的retriever进行retrieve
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        vector_nodes_large = self._vector_retriever_large.retrieve(query_bundle)

        #确保集合中的节点id对应的相似度由大到小排列
        vector_ids = {n.node_id for n in sorted(vector_nodes,key=lambda node: node.score,reverse=True)} 
        # vector_ids_large = {n.node_id for n in vector_nodes_large}

        #对于更大范围的相似度索引结果，取出其中含有关键词的节点
        keyword_ids = []
        for n in sorted(vector_nodes_large,key=lambda node: node.score,reverse=True):
            for k in self.keywords:
                if(k in word_tokenize(n.get_content())):
                    #判断关键词是否在文章片段的分词结果中
                    keyword_ids.append(n.node_id)
                    break

        combined_dict = {n.node_id: n for n in vector_nodes}
        combined_dict.update({n.node_id: n for n in vector_nodes_large if n.node_id in keyword_ids })

        #合并两组节点
        if(keyword_ids==[]):
            #不含有关键词的情况下照常进行相似度索引
            retrieve_ids = vector_ids
        else:
            keyword_ids_top=set(keyword_ids[:self.max_num_keyword_nodes]) #取相似度最高的几个关键词节点
            vector_ids_unique=vector_ids-keyword_ids_top  #top相似度集合中独有的节点
            retrieve_ids=keyword_ids_top #关键词集合和top相似度集合共有的节点+关键词集合中独有的节点
            add_num=len(vector_ids)-len(keyword_ids_top)
            retrieve_ids=set(list(vector_ids_unique)[:add_num]).union(retrieve_ids) #额外添加部分top相似度集合中独有的节点

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [14]:

from llama_index.retrievers.auto_merging_retriever import AutoMergingRetriever
base_retriever = base_index.as_retriever(similarity_top_k=6)
large_retriever = base_index.as_retriever(similarity_top_k=len(leaf_nodes)//2)

custom_retriever = CustomRetriever(base_retriever, large_retriever,max_num_keyword_nodes=3,keywords=[i.split(':')[0] for i in prompts[15][0].split('\n')])  
#创建混合索引实例
retriever = AutoMergingRetriever(custom_retriever, storage_context, verbose=True)

## 创建混合索引引擎

In [15]:
from llama_index.query_engine import RetrieverQueryEngine
custom_query_engine = RetrieverQueryEngine.from_args(retriever)

In [16]:
#修改默认的prompt内容
from llama_index.prompts import PromptTemplate
qa_prompt_tmpl_str = (
    """You are a biologist and patent expert. 
    You will be provided with some contents from a patent and will be asked to answer specific questions related to the patent. 
    Please answer the question only using the provided contents and do not make up the answer with prior knowledge."""
    "---------------------\n"
    "Content is :"
    "{context_str}\n"
    "---------------------\n"
    "Given the provided content, please answer the query:\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

custom_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

## 问题及回答

In [18]:
from IPython.display import Markdown, display
question1="Is this patent describing agents (e.g. molecules) for inhibiting Malassezia fungus? Such applications should be claimed in the “Claims” of the patent. Please make sure the patent is specific about inhibition of Malassezia. Finally, plesae answer 'Yes' or 'No' first, then explain the reason in next line." #Please focuse on the given context and don't use prior knowledge.
response = custom_query_engine.query(question1)
display(Markdown(f"<b>{response}</b>"))

<b>Yes.

The patent describes the use of certain peptides and salts thereof for inhibiting Malassezia fungus. Claim 153 specifically mentions the use of the peptides for inhibiting the growth of Malassezia spp., and claim 155 mentions the use of the peptides for inhibiting the growth of vancomycin-resistant Enterococcus faecium, which can be a Malassezia-like organism. Additionally, the patent describes the use of the peptides for inhibiting the formation of biofilms by Malassezia spp. (claim 156). Therefore, the patent is specific about inhibition of Malassezia.</b>

In [19]:
#当前问题prompt仅适用单个sequence回答
question2="""Given content describing the functions and applications of the patent.
        Please determine if sequence number "{0}" have the ability to inhibit Malassezia fungus. 
        Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses.
        Please focuse on the given context and don't use prior knowledge. You should focus on sequence number "{0}" and ignore other irrelvant sequence number!
        Finally, plesae do analysis and explain the reason first, then print "The answer is: 'Yes'" or "The answer is: 'No'" in next line.
            """.format(prompts[15][0])

response2=custom_query_engine.query(question2)
print(question2)
print('--------------------------------------------------------------------')
display(Markdown(f"<b>{response2}</b>"))

> Merging 2 nodes into parent node.
> Parent node id: a669df79-2028-4c93-a386-ab511941c0f0.
> Parent node text: To this solution will be added an exemplary peptide of SEQ_ID_NO_1, SEQ_ID_NO_2, SEQ_ID_NO_3, SEQ...

Given content describing the functions and applications of the patent.
        Please determine if sequence number "SEQ_ID_NO_7:VVRVVRRVVRVVRR" have the ability to inhibit Malassezia fungus. 
        Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses.
        Please focuse on the given context and don't use prior knowledge. You should focus on sequence number "SEQ_ID_NO_7:VVRVVRRVVRVVRR" and ignore other irrelvant sequence number!
        Finally, plesae do analysis and explain the reason first, then print "The answer is: 'Yes'" or "The answer is: 'No'" in next line.
            
--------------------------------------------------------------------


<b>Hello! I'm here to help you with your query based on the provided content of the patent. Please note that I will only answer based on the specific content provided and not use prior knowledge.

Based on the content provided, the target sequence number "SEQ_ID_NO_7:VVRVVRRVVRVVRR" is mentioned as a peptide that was screened for activity against Enterococcus faecium and Staphylococcus aureus. However, there is no mention of Malassezia fungus in the provided content.

Therefore, I cannot determine if sequence number "SEQ_ID_NO_7:VVRVVRRVVRVVRR" has the ability to inhibit Malassezia fungus based on the provided content.

The answer is: No.</b>

In [20]:
#输入prompt中的节点
#做embedding相似度匹配的只有query中的内容，并不涉及整个prompt
from llama_index.core.response.notebook_utils import display_source_node
for node in response2.source_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** 816f3b9e-958e-4c21-b70d-5bcfb5532681<br>**Similarity:** 0.8774757343130123<br>**Text:** 12. For SEQ_ID_NO_1, the majority of the results fall within a narrow range, in contrast to those of the comparators.TABLE 10DrugMIC rangeModeMIC50MIC90% R1EscherichiaSEQ_ID_NO_1 1-8224—coliCeftolozane -0.06->320.120.2514.9(n = 101)TazobactamPiperacillin - 0.25->12822648.9TazobactamMeropenem≤0.008-4   0.0150.0150.031Tobramycin0.25->640.50.53221.7Ceftazidime0.06->320.250.5>3233.6Colistin≤0.03-0.5  0.120.120.120Levofloxacin0.015->8 >84>846.5Table 11 summarizes the activity of SEQ_ID_NO_1 against the various resistance phenotypes in this study.<br>

--------


**Node ID:** 90475ac5-7caa-43a6-ba36-5d41b1259fd5<br>**Similarity:** 0.8679760286527705<br>**Text:** FIG. 6 shows the MIC distributions of SEQ_ID_NO_1 and the comparator drugs against the E. faecium isolates, with SEQ_ID_NO_1 displaying a fairly narrow distribution against the majority of isolatesTABLE 4DrugMIC rangeModeMIC50MIC90% R1EnterococcusSEQ_ID_NO_1≤0.12-4   112—faeciumLinezolid 1-32224 3.8(n = 104)Daptomycin0.06->32248—Vancomycin0.25->32>321>3247.1Doxycycline≤0.03-32  0.1241625  Trimethoprim -≤0.008->8    >8>8>8—SulfamethoxazoleClindamycin≤0.15->16 >16>16>16—Levofloxacin0.25->8 >8>8>878.8Against the collection of S. aureus isolates (Table 5; n=104),<br>

--------


**Node ID:** ee2bd62b-5caf-4881-bccf-cd70299c7c90<br>**Similarity:** 0.8659657075161271<br>**Text:** The plates were then inoculated with 10 μL of the inoculum resulting in a final cell density of approximately 5×105 CFU/mL, per well.Plates were stacked 3-4 high, covered with a lid on the top plate, placed in plastic bags, and incubated at 35° C. for 16 to 20 hrs. Following incubation, the microplates were removed from the incubator and viewed from the bottom using a plate viewer. For each date of assay, an uninoculated solubility control plate was observed for sterility and evidence of drug precipitation. The MIC was read and recorded as the lowest concentration of drug that inhibited visible growth of the organism.Results and DiscussionThe summary data for evaluating SEQ_ID_NO_1 against the ESKAPE pathogens and E. coli are shown in Tables 4-10 and FIGS. 6-12. The tables contain the MIC ranges, modes, MIC50 and MIC90 values, whereas the figures show the MIC distributions, for SEQ_ID_NO_1 and each comparator against Enterococcus faecium (Table 4 and FIG. 6), Staphylococcus aureus (Table 5 and FIG.<br>

--------


**Node ID:** a669df79-2028-4c93-a386-ab511941c0f0<br>**Similarity:** 0.8528482454292385<br>**Text:** To this solution will be added an exemplary peptide of SEQ_ID_NO_1, SEQ_ID_NO_2, SEQ_ID_NO_3, SEQ_ID_NO_4, SEQ_ID_NO_5, SEQ_ID_NO_6, SEQ_ID_NO_7, SEQ_ID_NO_8, SEQ_ID_NO_9, SEQ_ID_NO_10, SEQ_ID_NO_11, SEQ_ID_NO_12, or SEQ_ID_NO_13, diluted 2-fold in MHB. The resulting mixture will be incubated for about 1 hour. The well-associated biomass will be stained with crystal violet and quantified by measuring absorbance at 550 nm following solubilization in 30% acetic acid. Biofilm disruption will be quantitated by the absorbance at 550 nm as a function of the concentration of peptide.
Example 15: Administration of Compositions with a Biofilm Disrupter--P. aeruginosa or S. aureus cells will be grown in vinyl microtiter plates in Mueller-Hinton broth for 24 h at 37° C. to allow a mature biofilm to form. After about 24 hrs, a composition containing 0.1% w/w cysteamine, an exemplary peptide of SEQ_ID_NO_1, SEQ_ID_NO_2, SEQ_ID_NO_3, SEQ_ID_NO_4, SEQ_ID_NO_5, SEQ_ID_NO_6, SEQ_ID_NO_7, SEQ_ID_NO_8, SEQ_ID_NO_9, SEQ_ID_NO_10, SEQ_ID_NO_11, SEQ_ID_NO_12, or SEQ_ID_NO_13, and polysorbate 80 will be diluted 2-fold in MHB. The resulting mixture will be incubated for about 1 hour. The well-associated biomass will be stained with crystal violet and quantified by measuring absorbance at 550 nm following solubilization in 30% acetic acid. Biofilm disruption will be quantitated by the absorbance at 550 nm as a function of the concentration of peptide.While exemplary embodiments have been shown and described herein, it will be obvious to those skilled in the art that such embodiments are provided by way of example only. Numerous variations, changes, and substitutions will occur to those skilled in the art. It should be understood that various alternatives to the embodiments described herein may be employed.<br>

--------


**Node ID:** fe10450c-1bde-471c-b66e-3d3845fdc90d<br>**Similarity:** 0.8486942556562711<br>**Text:** Exemplary peptides screened include RRWVRRVRRVWRRVVRVVRRWVRR (SEQ_ID_NO_1); IRRRRRRIRRRRRR (SEQ_ID_NO_2); IRRRIRRIRRRIRRIRRRIRR (SEQ_ID_NO_3); IRRIIRRIRRIIRRIRRIIRR (SEQ_ID_NO_4); VWRWVRRVWRWVRRVWRWVRR (SEQ_ID_NO_5); VWRWVRRVWRWVRR (SEQ_ID_NO_6); VVRVVRRVVRVVRR (SEQ_ID_NO_7); and VVRVVRVVVRVVRVVVRVVRV (SEQ_ID_NO_8).Each peptide was formulated in PBS buffer prior to testing.<br>

--------
